Copyright 2020 Montvieux Ltd

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import requests
from io import BytesIO
import PIL.Image
from IPython.display import display,clear_output,HTML
from IPython.display import Image as DisplayImage
import base64
import json
from io import StringIO
import ipywidgets as widgets
import sys
import time
import imageio
import numpy as np
import io

from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy

from plark_game import classes
from gym_plark.envs import plark_env,plark_env_guided_reward,plark_env_top_left


from stable_baselines3 import DQN, PPO, A2C
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv

import helper 
import datetime
import os

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


%matplotlib inline
%load_ext autoreload
%autoreload 2




2023-04-26 14:00:36.943261: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcudart.so.12


In [2]:
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [3]:

env = plark_env.PlarkEnv()
check_env(env)

INFO:gym_plark.envs.plark_env:plark.kwargs :{'render_height': 250, 'render_width': 310, 'driving_agent': 'pelican'}
INFO:gym_plark.envs.plark_env:self.image_based :True
INFO:plark_game.classes.environment:Opening config from:/Components/plark-game/plark_game/game_config/10x10/balanced.json
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/pelicanAgent_3_buoys.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/PantherAgentIllegalMove.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/pelicanAgent_5_buoys.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/PelicanAgentIllegalMove.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/pantherAgent_move_north.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agent

In [4]:
env = plark_env_guided_reward.PlarkEnvGuidedReward()
check_env(env)

In [5]:
env = plark_env_top_left.PlarkEnvTopLeft()
check_env(env)

INFO:gym_plark.envs.plark_env:plark.kwargs :{'driving_agent': 'pelican', 'render_height': 250, 'render_width': 310}
INFO:gym_plark.envs.plark_env:self.image_based :True
INFO:plark_game.classes.environment:Opening config from:/Components/plark-game/plark_game/game_config/10x10/balanced.json
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/pelicanAgent_3_buoys.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/PantherAgentIllegalMove.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/pelicanAgent_5_buoys.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/PelicanAgentIllegalMove.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/pantherAgent_move_north.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agent

In [6]:

# Define and Train the agent
#model = A2C('CnnPolicy', env).learn(total_timesteps=1000)

# Instantiate the env
env = plark_env.PlarkEnv(config_file_path='/Components/plark-game/plark_game/game_config/30x30/balanced.json')

n_eval_episodes = 2
training_steps = 10

model = PPO('CnnPolicy', env)
#model = PPO2('CnnPolicy', 'plark-v0') 
#model = A2C('CnnPolicy', env)
model.learn(training_steps)

print("****** STARTING EVALUATION *******")
mean_reward, n_steps = evaluate_policy(model, env, n_eval_episodes=n_eval_episodes, deterministic=False, render=False, callback=None, reward_threshold=None, return_episode_rewards=False)
print("****** EVALUATION FINISHED *******")
print("Mean Reward is " + str(mean_reward))
print("Number of steps is " + str(n_steps))

INFO:gym_plark.envs.plark_env:plark.kwargs :{'render_height': 250, 'render_width': 310, 'driving_agent': 'pelican'}
INFO:gym_plark.envs.plark_env:self.image_based :True
INFO:gym_plark.envs.plark_env:config filepath: /Components/plark-game/plark_game/game_config/30x30/balanced.json
INFO:plark_game.classes.environment:Opening config from:/Components/plark-game/plark_game/game_config/30x30/balanced.json
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/pelicanAgent_3_buoys.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/PantherAgentIllegalMove.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/pelicanAgent_5_buoys.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/PelicanAgentIllegalMove.py
INFO:plark_game.classes.newgame:Opening agent from:/Components/plark-game/plark_game/agents/basic/pant

****** STARTING EVALUATION *******


/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


****** EVALUATION FINISHED *******
Mean Reward is -1.5500000156462193
Number of steps is 0.3500000052154064


In [7]:
#Save model 
basicdate = str(datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))

basepath = '/data/agents/models'
exp_name = 'test_' + basicdate
exp_path = os.path.join(basepath, exp_name)

print(exp_path)


/data/agents/models/test_20230426_140625


In [8]:



modeltype = 'PPO'
modelplayer = env.driving_agent 
render_height = env.render_height
render_width = env.render_width

helper.save_model(exp_path,model,modeltype,modelplayer,render_height,render_width,basicdate)


INFO:helper:Checking folder: /data/agents/models/test_20230426_140625/PPO_20230426_140625_pelican
INFO:helper:Saving Metadata
INFO:helper:json saved to: /data/agents/models/test_20230426_140625/PPO_20230426_140625_pelican/metadata.json
INFO:helper:Saving Model
INFO:helper:model_dir: /data/agents/models/test_20230426_140625/PPO_20230426_140625_pelican
INFO:helper:model_path: /data/agents/models/test_20230426_140625/PPO_20230426_140625_pelican/PPO_20230426_140625_pelican.zip


('/data/agents/models/test_20230426_140625/PPO_20230426_140625_pelican/PPO_20230426_140625_pelican.zip',
 '/data/agents/models/test_20230426_140625/PPO_20230426_140625_pelican',
 'PPO_20230426_140625_pelican')

In [9]:

helper.save_model_with_env_settings(exp_path,model,modeltype,env,basicdate)
    

INFO:helper:Checking folder: /data/agents/models/test_20230426_140625/PPO_20230426_140625_pelican
INFO:helper:Saving Metadata
INFO:helper:json saved to: /data/agents/models/test_20230426_140625/PPO_20230426_140625_pelican/metadata.json
INFO:helper:Saving Model
INFO:helper:model_dir: /data/agents/models/test_20230426_140625/PPO_20230426_140625_pelican
INFO:helper:model_path: /data/agents/models/test_20230426_140625/PPO_20230426_140625_pelican/PPO_20230426_140625_pelican.zip


('/data/agents/models/test_20230426_140625/PPO_20230426_140625_pelican/PPO_20230426_140625_pelican.zip',
 '/data/agents/models/test_20230426_140625/PPO_20230426_140625_pelican',
 'PPO_20230426_140625_pelican')

# Make a video

In [10]:
video_path = '/test.mp4'
basewidth,hsize = helper.make_video(model,env,video_path)

In [11]:
video = io.open(video_path, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" width="'''+str(basewidth)+'''" height="'''+str(hsize)+'''" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))